# Training and saving a model

This script demonstrates how to create and save a model.

## Set configuration

The methods and modules in this example make use of information from the `configure.json` file, which is located in the `/doc` folder. Fill in the blanks in the file. Some of them are: the model type, the number of nodes, the number of epochs in the training step, the size of the data test, the size of the windowing data, checking if the data contains accumulated values, and a flag to apply the moving average.

In [1]:
configure_json = open('../doc/configure.json', 'r')

import json
print(json.dumps(json.load(configure_json), indent=4))

{
    "ncovid": "ML COVID-19 configure file",
    "author": "NatalNet NCovid",
    "published_at": 2021,
    "folder_configs": {
        "docs_path": "../doc/",
        "data_path": "../dbs/",
        "model_path": "fitted_model/",
        "model_path_remote": "https://",
        "glossary_file": "glossary.json"
    },
    "model_configs": {
        "type_used": "Artificial",
        "is_predicting": "False",
        "Artificial": {
            "model": "lstm",
            "nodes": 300,
            "epochs": 100,
            "dropout": 0.1,
            "batch_size": 64,
            "earlystop": 30,
            "is_output_in_input": "True",
            "data_configs": {
                "is_accumulated_values": "False",
                "is_apply_moving_average": "True",
                "window_size": 7,
                "data_test_size_in_days": 35,
                "type_norm": ""
            },
            "Autoregressive": {
                "model": "arima",
                "p": 1,
    

To load this set of configurations, import the configs_manner.py file.

In [2]:
import sys
sys.path.append("../src")

import configs_manner

print("Model type: \n", configs_manner.model_type)
print("\n")
print("Model subtype: \n", configs_manner.model_subtype)
print("\n")
print("Number of model nodes: \n", configs_manner.model_infos["model_nodes"])
print("\n")
print("Model window size: \n", configs_manner.model_infos["model_epochs"])

Model type: 
 Artificial


Model subtype: 
 lstm


Number of model nodes: 
 300


Model window size: 
 100


To configure any data or model param, just change the value in the `configure.json` file.

## Build data

To build a model, you must first collect and create a dataset, from which some information will be used to automatically build the model architecture. We're making a data request from a remote location.

In [3]:
# specif code to the remote repository data.
repo = "p971074907"
# coutry and state acronym splited by a ":"
path = "brl:rn"
# columns (or features) to be extracted from the database, each one splited by a ":"
feature = "date:newDeaths:newCases:"
# start date for the data request.
begin = "2020-05-01"
# finish date for the data request.
end = "2021-07-01"

# import the data_manner.py file. (taking into account that you are in src/ path)
import data_manner

# creating the DataConstructor instance
data_constructor = data_manner.DataConstructor()
# collect data from the remote repository.
collected_data = data_constructor.collect_dataframe(path, repo, feature, begin, end)

# building the data and train to set up the number of features to model model archtecture.
train = data_constructor.build_train(collected_data)

## Create model

The Artificial model, bound on the Artificial Neural Networks subclass, will be used in this script. It will be the well-known Long-Short Term Memory ANN that is used (LSTM). Almost all procedures in our modules make use of class objects and class methods. To begin creating a model constructor, import the manner associated with the desired mode. In this case, the `lstm manner.py` script is used.

In [4]:
# import the lstm_manner.py file. (taking into account that you are in src/ path)
from models.artificial import lstm_manner

Create a model manner instance using `ModelLSTM()`, and build the architecture using the `creation()` method.

In [5]:
# creating  a model instance
lstm_model = lstm_manner.ModelLSTM(path)
# set up the model architecture
lstm_model.creating()
print(lstm_model)


LSTM Model
	input, output and timesteps: 7
	lstm nodes: 300
	features: 2
	dropout: 0.1


In [6]:
lstm_model.model

## Fit model

To train the instantiated model, call the `fitting()` model method and pass the train inputs (`train.x`) and the train targets (`train.y`) as the function args.

In [7]:
print("This can take some time.")
lstm_model.fiting(train.x, train.y, verbose=1)
print("LSTM model trained!")

This can take some time.
Epoch 1/100
7/7 [==============================] - 0s 44ms/step - loss: 2332.4890
Epoch 2/100
7/7 [==============================] - 0s 41ms/step - loss: 622.6501
Epoch 3/100
7/7 [==============================] - 0s 44ms/step - loss: 361.3294
Epoch 4/100
7/7 [==============================] - 0s 41ms/step - loss: 443.9677
Epoch 5/100
7/7 [==============================] - 0s 41ms/step - loss: 303.9600
Epoch 6/100
7/7 [==============================] - 0s 44ms/step - loss: 238.3689
Epoch 7/100
7/7 [==============================] - 0s 42ms/step - loss: 168.1460
Epoch 8/100
7/7 [==============================] - 0s 40ms/step - loss: 144.2268
Epoch 9/100
7/7 [==============================] - 0s 41ms/step - loss: 112.0583
Epoch 10/100
7/7 [==============================] - 0s 44ms/step - loss: 124.7233
Epoch 11/100
7/7 [==============================] - 0s 41ms/step - loss: 528.4888
Epoch 12/100
7/7 [==============================] - 0s 39ms/step - loss: 128.8172

## Save model

To save the model after it has been trained, use the `saving()` method. The save method does not accept a name or any arguments. It will create a unique id for the model in the `dbs/` folder.

In [8]:
lstm_model.saving()

This method will create two files: `metadata.json` and `.h5`. The `metadata.json` file will contain the model information as well as a UUID (unique identifier) that corresponds to the `.h5` file name.